In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('filled_missings_df.csv')

In [4]:
df.head()

,Unnamed: 0,ListingKey,ListingNumber,ListingCreationDate,Term,LoanStatus,BorrowerAPR,BorrowerRate,LenderYield,ListingCategory (numeric),...,LP_CollectionFees,LP_GrossPrincipalLoss,LP_NetPrincipalLoss,LP_NonPrincipalRecoverypayments,PercentFunded,Recommendations,InvestmentFromFriendsCount,InvestmentFromFriendsAmount,Investors,Defaulted
0,0,1021339766868145413AB3B,193129,13751.0,36,Completed,0.16516,0.1580,0.1380,0,...,0.0,0.0,0.0,0.0,1.0,0,0,0.0,258,0
1,1,10273602499503308B223C1,1209647,16128.0,36,Current,0.12016,0.0920,0.0820,2,...,0.0,0.0,0.0,0.0,1.0,0,0,0.0,1,0
2,2,0EE9337825851032864889A,81716,13518.0,36,Completed,0.28269,0.2750,0.2400,0,...,0.0,0.0,0.0,0.0,1.0,0,0,0.0,41,0
3,3,0EF5356002482715299901A,658116,15635.0,36,Current,0.12528,0.0974,0.0874,16,...,0.0,0.0,0.0,0.0,1.0,0,0,0.0,158,0
4,4,0F023589499656230C5E3E2,909464,15962.0,36,Current,0.24614,0.2085,0.1985,2,...,0.0,0.0,0.0,0.0,1.0,0,0,0.0,20,0


In [5]:
df.columns

Index(['Unnamed: 0', 'ListingKey', 'ListingNumber', 'ListingCreationDate',
       'Term', 'LoanStatus', 'BorrowerAPR', 'BorrowerRate', 'LenderYield',
       'ListingCategory (numeric)', 'BorrowerState', 'EmploymentStatus',
       'EmploymentStatusDuration', 'IsBorrowerHomeowner', 'CurrentlyInGroup',
       'DateCreditPulled', 'CreditScoreRangeLower', 'CreditScoreRangeUpper',
       'FirstRecordedCreditLine', 'OpenCreditLines',
       'TotalCreditLinespast7years', 'OpenRevolvingAccounts',
       'OpenRevolvingMonthlyPayment', 'InquiriesLast6Months', 'TotalInquiries',
       'CurrentDelinquencies', 'AmountDelinquent', 'DelinquenciesLast7Years',
       'PublicRecordsLast10Years', 'PublicRecordsLast12Months',
       'RevolvingCreditBalance', 'BankcardUtilization',
       'AvailableBankcardCredit', 'TotalTrades',
       'TradesNeverDelinquent (percentage)', 'TradesOpenedLast6Months',
       'DebtToIncomeRatio', 'IncomeRange', 'IncomeVerifiable',
       'StatedMonthlyIncome', 'LoanKey', 'Loa

### I think all those should be droppedd if we assume that we are making a model focused on scoring clients before giving them a loan

In [8]:
cols_to_drop = ['Unnamed: 0', 'ListingKey', 'ListingNumber', 'LoanStatus', 'LoanKey', 'LoanCurrentDaysDelinquent', 'LoanNumber', 'MemberKey',
               'LP_CustomerPayments', 'LP_CustomerPrincipalPayments', 'LP_InterestandFees', 'LP_ServiceFees', 'LP_CollectionFees', 'LP_GrossPrincipalLoss',
               'LP_NetPrincipalLoss', 'LP_NonPrincipalRecoverypayments']

In [9]:
df = df.drop(columns=cols_to_drop)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114805 entries, 0 to 114804
Data columns (total 47 columns):
 #   Column                              Non-Null Count   Dtype  
---  ------                              --------------   -----  
 0   ListingCreationDate                 114413 non-null  float64
 1   Term                                114805 non-null  int64  
 2   BorrowerAPR                         114805 non-null  float64
 3   BorrowerRate                        114805 non-null  float64
 4   LenderYield                         114805 non-null  float64
 5   ListingCategory (numeric)           114805 non-null  int64  
 6   BorrowerState                       114805 non-null  object 
 7   EmploymentStatus                    114805 non-null  object 
 8   EmploymentStatusDuration            114805 non-null  float64
 9   IsBorrowerHomeowner                 114805 non-null  int64  
 10  CurrentlyInGroup                    114805 non-null  int64  
 11  DateCreditPulled          

In [12]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.2, random_state=101)

#### Binning with optbin

In [16]:
from optbinning import OptimalBinning

y_train = train_df["Defaulted"]
X_train = train_df.drop(columns=["Defaulted"])
X_test = test_df.copy()

binning_transformers = {}
X_train_binned = pd.DataFrame()
X_test_binned = pd.DataFrame()
X_train_binned_woe = pd.DataFrame()
X_test_binned_woe = pd.DataFrame()

# Just loop
for column in X_train.columns:
    # Check for dtype
    dtype = "numerical" if pd.api.types.is_numeric_dtype(X_train[column]) else "categorical"

    # fit on train
    optb = OptimalBinning(name=column, dtype=dtype, solver="cp")
    optb.fit(X_train[column], y_train)

    # Store in dict
    binning_transformers[column] = optb

    # Transform to WOE
    X_train_binned_woe[column] = optb.transform(X_train[column], metric="woe")
    X_test_binned_woe[column] = optb.transform(X_test[column], metric="woe")

    # Transform to bin edges
    X_train_binned[column] = optb.transform(X_train[column], metric="bins")
    X_test_binned[column] = optb.transform(X_test[column], metric="bins")

### Feature Selection

### I would try 
1. Information Value
2. Lasso regression
3. Maybe random forest and based on feature importance
4. Also maybe combine 1 and 2 and 3, so if IV is high (maybe above ~ 0.03) and coefficient non zero and decent feature importance 